A data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. The task is to predict whether a given blight ticket will be paid on time.

All data has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/).

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

Two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric is the Area Under the ROC Curve (AUC). 


___

Create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [35]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
#     load training data
    df_train = pd.read_csv('train.csv', encoding = 'ISO-8859-1' )
    
#     ignore the NaN value in compliance
    df_train = df_train[(df_train['compliance'] == 0) | (df_train['compliance'] == 1)]
    df_test = pd.read_csv('test.csv')
    
#     load address data
    df_address = pd.read_csv('addresses.csv')
    df_latlons = pd.read_csv('latlons.csv')
    
#     join 2 dataframes to add latlon to training and testing data
    address = df_address.set_index('address').join(df_latlons.set_index('address'), how='left')
    train = df_train.set_index('ticket_id').join(address.set_index('ticket_id'))
    test = df_test.set_index('ticket_id').join(address.set_index('ticket_id'))
    
#     remove the values that only exists in training set
    train_drop = ['payment_amount', 'payment_date', 'payment_status', 'balance_due',
              'collection_status', 'compliance_detail']
    train = train.drop(train_drop, axis = 1)
    
#     remove the string values that cannot be learned
    string_drop = ['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition',  
               'judgment_amount', 'grafitti_status']
#     keeped features = 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost'
    train = train.drop(string_drop, axis = 1).dropna()
    test = test.drop(string_drop, axis = 1).fillna(0)
    
#     no need to train_test_split, y_train is the label, X_test is the test and we don't have y_test
    y_train = train['compliance']
    X_train = train.drop('compliance', axis = 1)
    X_test = test
    
#     choose appropriate learning model
# 1.    SVC model
#     from sklearn.svm import SVC
#     clf = SVC().fit(X_train, y_train)
#     y_proba = clf.predict_proba(X_train)
#     test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
#     test_df['probability'] = y_proba
#     test_df.set_index('ticket_id', inplace=True)
    
# 2. gradient-boost trees
    from sklearn.ensemble import GradientBoostingClassifier
    clf = GradientBoostingClassifier().fit(X_train, y_train)
    y_proba = clf.predict_proba(X_test)[:,1]
#     test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
#     test['probability'] = y_proba
    result = pd.DataFrame(index = list(X_test.index), columns = ['probability'])
    result['probability'] = y_proba

    return result['probability']


In [36]:
# len(blight_model())

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


61001

In [16]:
# y_proba.shape


(60996,)

In [17]:
# # load training data
# import pandas as pd
# import numpy as np
# df_train = pd.read_csv('train.csv', encoding = 'ISO-8859-1' )
# # ignore the NaN value in compliance
# df_train = df_train[(df_train['compliance'] == 0) | (df_train['compliance'] == 1)]
# # df_train[list(df_train.columns)[14:]]

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
# df_test = pd.read_csv('test.csv')

In [19]:
# # load address data
# df_address = pd.read_csv('addresses.csv')
# df_latlons = pd.read_csv('latlons.csv')
# # join 2 dataframes to add latlon to training and testing data
# address = df_address.set_index('address').join(df_latlons.set_index('address'), how='left')
# train = df_train.set_index('ticket_id').join(address.set_index('ticket_id'))
# test = df_test.set_index('ticket_id').join(address.set_index('ticket_id'))

In [20]:
# train_drop = ['payment_amount', 'payment_date', 'payment_status', 'balance_due',
#               'collection_status', 'compliance_detail']
# # compliance [target variable for prediction] 
# train = train.drop(train_drop, axis = 1)


In [21]:
# string_drop = ['agency_name', 'inspector_name', 'violator_name',
#        'violation_street_number', 'violation_street_name',
#        'violation_zip_code', 'mailing_address_str_number',
#        'mailing_address_str_name', 'city', 'state', 'zip_code',
#        'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
#        'violation_code', 'violation_description', 'disposition',  
#                'judgment_amount', 'grafitti_status']
# # features = 'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost'

In [22]:
# train = train.drop(string_drop, axis = 1).dropna()
# # len(train)
# # 159880
# # len(train.dropna())
# # 159877
# # train

In [23]:
# test = test.drop(string_drop, axis = 1).dropna()
# # test


In [24]:
# # no need to train_test_split, y_train is the label, X_test is the test and we don't have y_test
# y_train = train['compliance']
# X_train = train.drop('compliance', axis = 1)
# X_test = test

In [30]:
# in case to scale Features to Reduce Computation Time(Minmaxscaler)
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# X_train[X_train['zip_code'] == 'N9A2H9']
# result = pd.DataFrame(index = list(X_test.index), columns = ['probability'])
# result['probability'] = y_proba
# result

,probability
284932,0.054529
285362,0.021018
285361,0.058615
285338,0.083358
285346,0.089707
285345,0.083358
285347,0.084279
285342,0.334442
285530,0.030313
284989,0.035971


In [10]:
# # learning model
# from sklearn.svm import SVC
# clf = SVC().fit(X_train, y_train)
# y_proba = clf.predict_proba(X_train)
# test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
# test_df['probability'] = y_proba
# test_df.set_index('ticket_id', inplace=True)
# test_df['probability']
# # proba_list = clf.predict_proba(X_test)
# # from sklearn.metrics import roc_curve, auc
# # test_score = clf.decision_function(X_test)
# # fpr, tpr, _ = roc_curve(y_test, test_score)
# # auc = auc(fpr, tpr)
# # auc


AttributeError: predict_proba is not available when  probability=False